# Project 2


# Movie Genre Classification

Classify a movie genre based on its plot.

<img src="moviegenre.png"
     style="float: left; margin-right: 10px;" />




https://www.kaggle.com/c/miia4200-20191-p2-moviegenreclassification/overview


### Data

Input:
- movie plot

Output:
Probability of the movie belong to each genre


### Evaluation

- 20% API
- 30% Create a solution using with a Machine Learning algorithm - Presentation (5 slides)
- 50% Performance in the Kaggle competition (Normalized acording to class performance in the private leaderboard)


### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.

See https://arxiv.org/abs/1702.01992

## Sample Submission

In [88]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer
import nltk
import xgboost
from sklearn.linear_model import LogisticRegression

In [3]:
import re

In [222]:
dataTraining = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

### Create count vectorizer


In [223]:

dataTraining['title']=dataTraining['title'].str.lower()
dataTraining['title']=dataTraining['title'].str.replace(',', '') 

dataTraining['title']=dataTraining['title'].str.replace('.', '') 
dataTraining['title']=dataTraining['title'].str.replace('-', '') 
dataTraining['title']=dataTraining['title'].str.replace('_', '') 


In [224]:
dataTraining['plot']=dataTraining['plot'].str.lower()
dataTraining['plot']=dataTraining['plot'].str.replace(',', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('.', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('-', '') 
dataTraining['plot']=dataTraining['plot'].str.replace('_', '') 

In [225]:
wordnet_lemmatizer = WordNetLemmatizer()

In [226]:
dataTraining['uno']= dataTraining['title']+dataTraining['plot']

In [228]:
wordnet_lemmatizer = WordNetLemmatizer()
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word, 's') for word in words]

vect = TfidfVectorizer(analyzer=split_into_lemmas, min_df=2,max_features=None,sublinear_tf=True,
                       strip_accents='unicode',ngram_range=(1,2),stop_words='english')

X_dtm = vect.fit_transform(dataTraining['uno'],dataTraining['year'])
X_dtm.shape

(7895, 21371)

### Create y

In [229]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [230]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

### Train multi-class multi-label model

In [21]:

#clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1,max_depth=10,n_estimators=1000, random_state=60,criterion='entropy'))

In [231]:
LogReg_Model = OneVsRestClassifier(LogisticRegression(multi_class='multinomial',solver='newton-cg',C=1.5,
                                                     warm_start=True,penalty='l2',random_state=42))


LogReg_Model.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=42, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=True),
          n_jobs=None)

In [232]:
y_pred_genres = LogReg_Model.predict_proba(X_test)

In [233]:
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.8878224926750639

### Predict the testing dataset

In [111]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = LogReg_Model.predict_proba(X_test_dtm)


In [112]:
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)

In [16]:
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.143030,0.101960,0.024454,0.029938,0.354552,0.138830,0.030787,0.490140,0.073159,0.101339,...,0.025069,0.063208,0.000000,0.362818,0.056648,0.008970,0.017522,0.202605,0.033989,0.018117
4,0.122624,0.085786,0.024213,0.084795,0.370949,0.216657,0.080359,0.515684,0.062976,0.067019,...,0.024734,0.060935,0.000477,0.149703,0.058190,0.014248,0.020099,0.204794,0.030438,0.018506
5,0.151364,0.110284,0.013762,0.075334,0.304837,0.448736,0.021010,0.611544,0.081741,0.169121,...,0.044538,0.261372,0.000000,0.335987,0.128505,0.001016,0.048658,0.423242,0.052693,0.025351
6,0.154448,0.125772,0.020991,0.064124,0.340779,0.140892,0.009133,0.632038,0.068287,0.063631,...,0.131074,0.088418,0.000000,0.197224,0.132208,0.001432,0.039743,0.269385,0.077607,0.017862
7,0.175143,0.210069,0.035476,0.032505,0.313850,0.243150,0.021793,0.427885,0.079781,0.143879,...,0.023859,0.090359,0.000048,0.205117,0.241663,0.002634,0.018403,0.259465,0.021569,0.017585


In [113]:
res.to_csv('pred_genres_text_RF5.csv', index_label='ID')